### PromptTemplate

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_teddynote import logging

logging.langsmith("edu-CH02-Prompt")

LangSmith 추적을 시작합니다.
[프로젝트명]
edu-CH02-Prompt


LLM 객체 정의

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

### 방법 1. from_template() 메소드로 PromptTemplate 생성

- 치환된 변수는 `{변수}`로 묶음

In [ ]:
from langchain_core.prompts import PromptTemplate

# template 정의
template = "{country} 의 수도는 어디인가요?"

# from_template 메소드를 사용하여 객체 생성
prompt = PromptTemplate.from_template(template)
prompt

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country} 의 수도는 어디인가요?')

In [5]:
prompt = prompt.format(country="대한민국")
prompt

'대한민국 의 수도는 어디인가요?'

In [ ]:
# template 정의
template = "{country} 의 수도는 어디인가요?"

# from_template 메소드를 사용하여 객체 생성
prompt = PromptTemplate.from_template(template)

# chain 생성
chain = prompt | llm

In [ ]:
# contry 변수에 입력된 값이 자동으로 치환되어 수정됨
# chain.invoke({"country": "대한민국"}).content
chain.invoke("대한민국").content

'대한민국의 수도는 서울입니다.'

### 방법 2. PromptTemplate 객체 생성 시에 prompt 생성

In [ ]:
# template 정의
template = "{country} 의 수도는 어디인가요?"

prompt = PromptTemplate(
    template=template,
    input_variables=["country"],
)

prompt

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country} 의 수도는 어디인가요?')

In [9]:
prompt.format(country="대한민국")

'대한민국 의 수도는 어디인가요?'

In [ ]:
# template 정의
template = "{country1}과 {country2}의 수도는 각각 어디인가요?"

# prompt_template 생성
prompt = PromptTemplate(
    template=template,
    input_variables=["country1"],
    partial_variables={
        "country2": "미국",  # dictionary 형태로 partial_variables 전달
    },
)

prompt

PromptTemplate(input_variables=['country1'], input_types={}, partial_variables={'country2': '미국'}, template='{country1}과 {country2}의 수도는 각각 어디인가요?')

In [12]:
prompt.format(country1="대한민국")

'대한민국과 미국의 수도는 각각 어디인가요?'

In [15]:
prompt = PromptTemplate.from_template(template)
prompt

PromptTemplate(input_variables=['country1', 'country2'], input_types={}, partial_variables={}, template='{country1}과 {country2}의 수도는 각각 어디인가요?')

In [16]:
prompt_partial = prompt.partial(country2="캐나다")
prompt_partial

PromptTemplate(input_variables=['country1'], input_types={}, partial_variables={'country2': '캐나다'}, template='{country1}과 {country2}의 수도는 각각 어디인가요?')

In [17]:
prompt_partial.format(country1="대한민국")

'대한민국과 캐나다의 수도는 각각 어디인가요?'

In [18]:
chain = prompt_partial | llm

In [20]:
chain.invoke("대한민국").content

'대한민국의 수도는 서울이고, 캐나다의 수도는 오타와입니다.'

In [ ]:
chain.invoke({"country1": "대한민국", "country2": "호주"}).content

'대한민국의 수도는 서울이고, 호주의 수도는 캔버라입니다.'

### `partial_variables` : 부분 변수 채움

In [22]:
from datetime import datetime

# 오늘 날짜 출력
datetime.now().strftime("%B %d")

'March 26'

In [ ]:
# 날짜 변환 함수 정의
def get_today():
    return datetime.now().strftime("%B %d")

In [ ]:
prompt = PromptTemplate(
    template="오늘의 날짜는 {today} 입니다. 오늘이 생일인 유명인 {n}명을 나열해 주세요. 생년월일을 표기해주세요.",
    input_variables=["n"],
    partial_variables={"today": get_today},
)

In [ ]:
# prompt 생성
prompt.format(n=3)

'오늘의 날짜는 March 26 입니다. 오늘이 생일인 유명인 3명을 나열해 주세요. 생년월일을 표기해주세요.'

In [ ]:
# chain 생성
chain = prompt | llm

In [27]:
# chain 실행 후 결과 출력
print(chain.invoke(3).content)

1. Keira Knightley (1985년 3월 26일)
2. Leonard Nimoy (1931년 3월 26일)
3. Diana Ross (1944년 3월 26일)


In [28]:
print(chain.invoke({"today": "Jan 02", "n": 3}).content)

1. Kate Middleton - 1982년 1월 9일
2. Taye Diggs - 1971년 1월 2일
3. Tia Carrere - 1967년 1월 2일


### 파일로부터 template 읽어오기

In [29]:
from langchain_core.prompts import load_prompt

prompt = load_prompt("prompts/fruit_color.yaml", encoding="utf-8")
prompt

PromptTemplate(input_variables=['fruit'], input_types={}, partial_variables={}, template='{fruit}의 색깔이 뭐야?')

In [30]:
prompt.format(fruit="사과")

'사과의 색깔이 뭐야?'

In [31]:
prompt2 = load_prompt("prompts/capital.yaml")
print(prompt2.format(country="대한민국"))

대한민국의 수도에 대해서 알려주세요.
수도의 특징을 다음의 양식에 맞게 정리해 주세요.
300자 내외로 작성해 주세요.
한글로 작성해 주세요.
----
[양식]
1. 면적
2. 인구
3. 역사적 장소
4. 특산품

#Answer:



In [34]:
from langchain_core.output_parsers import StrOutputParser
from langchain_teddynote.messages import stream_response

In [35]:
chain = prompt2 | ChatOpenAI(model_name="gpt-4o", temperature=0) | StrOutputParser()

In [36]:
answer = chain.stream({"country": "대한민국"})
stream_response(answer)

1. 면적: 대한민국의 수도인 서울특별시는 약 605.21㎢의 면적을 가지고 있습니다. 이는 전국에서 가장 큰 도시 중 하나로, 다양한 행정구역과 자연경관을 포함하고 있습니다.

2. 인구: 서울의 인구는 약 950만 명으로, 대한민국에서 가장 인구가 많은 도시입니다. 다양한 문화와 경제 활동이 활발하게 이루어지는 중심지입니다.

3. 역사적 장소: 서울에는 경복궁, 창덕궁, 덕수궁 등 조선시대의 궁궐들이 있으며, 이외에도 한양도성, 종묘 등 유네스코 세계문화유산으로 지정된 역사적 장소들이 많습니다.

4. 특산품: 서울은 전통과 현대가 조화를 이루는 도시로, 한복, 한지 공예품, 전통 음식인 김치와 떡 등이 유명합니다. 또한, 현대적인 패션과 기술 제품도 서울의 특산품으로 꼽힙니다.

### ChatPromptTemplate

- `system`
- `Human`
- `ai`

In [40]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_template("{country}의 수도는 어디인가요?")
chat_prompt

ChatPromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?'), additional_kwargs={})])

In [41]:
chat_prompt.format(country="대한민국")

'Human: 대한민국의 수도는 어디인가요?'

In [50]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
  [
    # role, message
    ("system", "당신은 친절한 AI 어시스턴트입니다. 당신의 이름은 {name} 입니다."),
    ("human", "반가워요!"),
    ("ai", "안녕하세요! 무엇을 도와드릴까요?"),
    ("human", "{user_input}"),
  ]
)

# chat message 생성
messages = chat_template.format_messages(
  name="테디", user_input="당신의 이름은 무엇입니까?"
)
messages

[SystemMessage(content='당신은 친절한 AI 어시스턴트입니다. 당신의 이름은 테디 입니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='반가워요!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='안녕하세요! 무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='당신의 이름은 무엇입니까?', additional_kwargs={}, response_metadata={})]

In [51]:
llm.invoke(messages).content

'제 이름은 테디입니다. 필요하신 도움이 있으신가요?'

In [53]:
chain = chat_template | llm 

In [54]:
chain.invoke({"name": "Teddy", "user_input": "당신의 별명은 무엇입니까?"}).content

'제 별명은 Teddy입니다. 어떻게 도와드릴까요?'

### MessagePlaceholder

아직 확정되지 않은 메시지나 키 값을 나중에 넣어주기 위한 위치 지정 방식 (키 필수)

In [63]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

chat_prompt = ChatPromptTemplate.from_messages(
  [
    # (
    #   "system",
    #   "당신은 요약 전문 AI 어시스턴트입니다. 당신의 임무는 주요 키워드로 대화를 요약하는 것입니다.",
    # ),
    MessagesPlaceholder(variable_name="conversation"),
    ("human", "지금까지의 대화를 {word_count} 단어로 요약합니다."),
  ]
)
chat_prompt

ChatPromptTemplate(input_variables=['conversation', 'word_count'], input_types={'conversation': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annota

In [64]:
formatted_chat_prompt = chat_prompt.format(
  word_count=5,
  conversation=[
    ("human", "안녕하세요! 저는 오늘 새로 입사한 테디입니다. 만나서 반갑습니다."),
    ("ai", "반가워요! 앞으로 잘 부탁 드립니다."),
  ]
)

print(formatted_chat_prompt)

Human: 안녕하세요! 저는 오늘 새로 입사한 테디입니다. 만나서 반갑습니다.
AI: 반가워요! 앞으로 잘 부탁 드립니다.
Human: 지금까지의 대화를 5 단어로 요약합니다.


In [65]:
# chain 생성
chain = chat_prompt | llm | StrOutputParser()

In [67]:
# chain 실행 및 결과 확인
chain.invoke(
  {
    "word_count": 5, 
    "conversation": [
      ("human", "안녕하세요! 저는 오늘 새로 입사한 리사 입니다. 만나서 반갑습니다."),
      ("ai", "반가워요! 앞으로 잘 부탁 드립니다.")
    ]
  }
)

'리사 새로 입사한 직원.'